# Notebook for Autogen Sample
The following code is example on how to use Autogen.
Version: 0.4.7

In [ ]:
!pip install "autogen-ext[magentic-one]" azure-identity aiohttp

### Sample Code for Autogen
#### Sample 1: Use Autogen to write code without human input.

Load environment variables from .env.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

Define the agent client using Azure OpenAI. Developer can choose other client such as OpenAI, or open source model.
Refer to the [documentation](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/components/model-clients.html) for more information.

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

## Define the client used for the agent
aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=os.getenv('AZURE_OPEN_AI_ENDPOINT'),
    model=os.getenv('AZURE_OPEN_AI_MODEL_NAME'),
    api_version="2024-12-01-preview",
    api_key=os.getenv('AZURE_OPEN_AI_KEY')
)

In [ ]:
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor


async def write_code():
    m1 = MagenticOne(client=aoai_client, code_executor=LocalCommandLineCodeExecutor())
    task = "Write an python code to calculate the nth fibonacci number, and also display the host name of the compute the codes being executed. Then, use the code to calculate the 10th fibonacci number and reply user."
    result = await Console(m1.run_stream(task=task))
    print(result)


await write_code()

#### Sample 2: Magentic-One with Human Input

Human in the loop for getting futher inputs.

In [ ]:
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor


async def write_code_hil():
    m1 = MagenticOne(client=aoai_client, code_executor=LocalCommandLineCodeExecutor(), hil_mode=True)
    task = "Write an python code to calculate the nth fibonacci number, and also display the host name of the compute the codes being executed. Then, ask user to provide the value of n and use the code to calculate the nth fibonacci number and reply user."
    result = await Console(m1.run_stream(task=task))
    print(result)


await write_code_hil()

#### Sample 3: Magentic-One with Azure Container Apps Dynamic Code Container

In [ ]:
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
token = credential.get_token("https://dynamicsessions.io/.default")
access_token = token.token

In [ ]:
from autogen_ext.code_executors.azure import ACADynamicSessionsCodeExecutor

code_executor = ACADynamicSessionsCodeExecutor(
    pool_management_endpoint=os.getenv('ACA_POOL_MANAGEMENT_ENDPOINT'),
    credential=credential
)

In [ ]:
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console


async def write_code_with_aca():
    m1 = MagenticOne(client=aoai_client, code_executor=code_executor)
    task = "Write an python code to calculate the nth fibonacci number, and also display the host name of the compute the codes being executed. Then, use the code to calculate the 10th fibonacci number and reply user."
    result = await Console(m1.run_stream(task=task))
    print(result)


await write_code_with_aca()